In [ ]:
import spacy
import random
import pandas as pd
import json

In [ ]:
from google.colab import drive
drive.mount("my-drive")

Mounted at my-drive


In [ ]:
df=pd.read_excel("my-drive/My Drive/annotations1.xlsx")
df1=pd.read_excel("my-drive/My Drive/facultyinfo1.xlsx")

In [ ]:
ids={"e_2":"Name","e_3":"Specialization","e_4":"Address","e_5":"Department","e_6":"Designation","e_7":"Degree","e_8":"Email","e_9":"Experience","e_10":"MobileNumber","e_13":"PersonalWebsite","e_14":"Qualification"}

training_data=[]

In [ ]:
def prepareTrainingData(df,df1,ids,training_data):
  for i in range(len(df1.index)):
    temp_dict={}
    temp_dict["entities"]=[]
    temp_list=[]
    temp_list.append(df1.iloc[i,0])
    data=json.loads(df.iloc[i,0])
    list1=data["entities"]
    
    for element in list1:
        ann_text=ids.get(element['classId'])
        if(ann_text==None):
            continue
        start=element['offsets'][0]['start']
        end=element['offsets'][0]['start']+len(element['offsets'][0]['text'])
        l1=[]
        l1.append(start)
        l1.append(end)
        l1.append(ann_text)
        l1=tuple(l1)
        temp_dict["entities"].append(l1)
    
    temp_list.append(temp_dict)
    training_data.append(tuple(temp_list))
  return training_data
# prepareTrainingData(df,df1,ids,training_data)

In [ ]:
training_data=prepareTrainingData(df,df1,ids,training_data)

In [ ]:
stdf=pd.read_excel('my-drive/My Drive/stanford annotation.xlsx')
stdf1=pd.read_excel('my-drive/My Drive/stanfordfacultyinfo.xlsx')

In [ ]:
ids1={"e_8":"Experience","e_3":"Address","e_2":"Specialization","e_7":"Email","e_5":"Designation","e_11":"Qualification","e_6":"Degree","e_9":"MobileNumber","e_10":"PersonalWebsite","e_4":"Department","e_1":"Name"}

In [ ]:
training_data1=[]
training_data1=prepareTrainingData(stdf,stdf1,ids1,training_data1)

In [ ]:
training_data.extend(training_data1)

In [ ]:
mitdf=pd.read_excel('my-drive/My Drive/mit annotation.xlsx')
mitdf1=pd.read_csv('my-drive/My Drive/mitfacultyinfo.csv')

In [ ]:
training_data2=[]
trainig_data2=prepareTrainingData(mitdf,mitdf1[:19],ids1,training_data2)

In [ ]:
training_data2

[('Polina  Anikeeva,Associate Professor in Materials Science and Engineering,Associate Professor in Brain and Cognitive Sciences,McGovern Institute for Brain Research,Associate Director, Research Laboratory of Electronics,B.S. in Biophysics, St. Petersburg State Polytechnic University, 2003,Ph.D. in Materials Science and Engineering, MIT, 2009,Room 36-849,anikeeva@mit.edu,Bioelectronics Group,CV,Bio; Biomaterials; Biophysics; Biotechnology; Electronic Materials; Magnetic Materials; Materials Chemistry; Implants; Nanotechnology; Photonic Materials,Research,Prof. Polina Anikeeva joined DMSE in 2011. Dr. Anikeeva\'s Ph.D. thesis focussed on physical properties and design of light emitting devices based on organic materials and nanoparticles, working under the supervision of Prof. Vladimir Bulovic in EECS. She previously held the Dean\'s Postdoctoral Fellowship, School of Medicine, Stanford and was in the group of Prof. Karl Deisseroth in the Department of Bioengineering. Her current resea

In [ ]:
len(training_data)

68

In [ ]:
for tp in training_data:
  dic=tp[1]
  prevend=-1
  for tp1 in dic['entities']:
    start=tp1[0]
    end=tp1[1]
    if prevend>=start:
      print('Overlapping in '+tp[0][:10])
    prevend=end


In [ ]:
for tp in training_data2:
  dic=tp[1]
  prevend=-1
  for tp1 in dic['entities']:
    start=tp1[0]
    end=tp1[1]
    if prevend>=start:
      print('Overlapping in '+tp[0][:10])
    prevend=end

Overlapping in Jeffrey C.


In [ ]:
training_data.extend(training_data2)

In [ ]:
def train_model(data,iterations):
  TRAIN_DATA=data
  nlp=spacy.blank('en')

  if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

  for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
      ner.add_label(ent[2])

  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
  with nlp.disable_pipes(*other_pipes):  
    optimizer = nlp.begin_training()

    for itn in range(iterations):
      random.shuffle(TRAIN_DATA)
      losses = {}

      for text, annotations in TRAIN_DATA:
        nlp.update(
                    [text],  
                    [annotations],  
                    drop=0.2,  
                    sgd=optimizer,  
                    losses=losses)
  return nlp

In [ ]:
nlp_model=train_model(training_data,25)

In [ ]:
nlp_model.to_disk('my-drive/My Drive/nlp_model')